<a href="https://colab.research.google.com/github/wendy60/Hybrid-recommender-system/blob/initial-user-interest-model/DKN_Deep_Knowledge_Aware_Network_for_News_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##Microsoft News Dataset (MIND)

# **Global settings and imports**

In [ ]:
pip install scrapbook

In [ ]:
pip install recommenders

In [ ]:
pip install tensorflow-gpu==1.15.2

In [ ]:
import sys
import os
from tempfile import TemporaryDirectory
import scrapbook as sb
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources, prepare_hparams
from recommenders.models.deeprec.models.dkn import DKN
from recommenders.models.deeprec.io.dkn_iterator import DKNTextIterator

print(f"System version: {sys.version}")
print(f"Tensorflow version: {tf.__version__}")

System version: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
Tensorflow version: 1.15.2


# **Download and load data**

In [ ]:
tmpdir = TemporaryDirectory()
data_path = os.path.join(tmpdir.name, "mind-demo-dkn")

yaml_file = os.path.join(data_path, r'dkn.yaml')
train_file = os.path.join(data_path, r'train_mind_demo.txt')
valid_file = os.path.join(data_path, r'valid_mind_demo.txt')
test_file = os.path.join(data_path, r'test_mind_demo.txt')
news_feature_file = os.path.join(data_path, r'doc_feature.txt')
user_history_file = os.path.join(data_path, r'user_history.txt')
wordEmb_file = os.path.join(data_path, r'word_embeddings_100.npy')
entityEmb_file = os.path.join(data_path, r'TransE_entity2vec_100.npy')
contextEmb_file = os.path.join(data_path, r'TransE_context2vec_100.npy')
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/deeprec/', tmpdir.name, 'mind-demo-dkn.zip')


100%|██████████| 11.3k/11.3k [00:00<00:00, 13.2kKB/s]



# **Create hyper-parameters**

In [ ]:

epochs = 10
history_size = 50
batch_size = 100

In [ ]:
hparams = prepare_hparams(yaml_file,
                          news_feature_file = news_feature_file,
                          user_history_file = user_history_file,
                          wordEmb_file=wordEmb_file,
                          entityEmb_file=entityEmb_file,
                          contextEmb_file=contextEmb_file,
                          epochs=epochs,
                          history_size=history_size,
                          batch_size=batch_size)
print(hparams)

kg_file=None,user_clicks=None,FEATURE_COUNT=None,FIELD_COUNT=None,data_format=dkn,PAIR_NUM=None,DNN_FIELD_NUM=None,n_user=None,n_item=None,n_user_attr=None,n_item_attr=None,iterator_type=None,SUMMARIES_DIR=None,MODEL_DIR=None,wordEmb_file=/tmp/tmpx7kiklp4/mind-demo-dkn/word_embeddings_100.npy,entityEmb_file=/tmp/tmpx7kiklp4/mind-demo-dkn/TransE_entity2vec_100.npy,contextEmb_file=/tmp/tmpx7kiklp4/mind-demo-dkn/TransE_context2vec_100.npy,news_feature_file=/tmp/tmpx7kiklp4/mind-demo-dkn/doc_feature.txt,user_history_file=/tmp/tmpx7kiklp4/mind-demo-dkn/user_history.txt,use_entity=True,use_context=True,doc_size=10,history_size=50,word_size=12600,entity_size=3987,entity_dim=100,entity_embedding_method=None,transform=True,train_ratio=None,dim=100,layer_sizes=[300],cross_layer_sizes=None,cross_layers=None,activation=['sigmoid'],cross_activation=identity,user_dropout=False,dropout=[0.0],attention_layer_sizes=100,attention_activation=relu,attention_dropout=0.0,model_type=dkn,method=classification

# **Train the DKN model**

In [ ]:

model = DKN(hparams, DKNTextIterator)

In [ ]:

print(model.run_eval(valid_file))

{'auc': 0.4641, 'group_auc': 0.4607, 'mean_mrr': 0.1394, 'ndcg@5': 0.1309, 'ndcg@10': 0.1793}


In [ ]:
model.fit(train_file, valid_file)

at epoch 1
train info: logloss loss:0.6839394362296088
eval info: auc:0.5373, group_auc:0.5104, mean_mrr:0.1706, ndcg@10:0.2349, ndcg@5:0.1716
at epoch 1 , train time: 375.8 eval time: 60.6
at epoch 2
train info: logloss loss:0.6197499130236901
eval info: auc:0.5405, group_auc:0.4909, mean_mrr:0.167, ndcg@10:0.2273, ndcg@5:0.1657
at epoch 2 , train time: 374.6 eval time: 60.6
at epoch 3
train info: logloss loss:0.5857534601779307
eval info: auc:0.5567, group_auc:0.5098, mean_mrr:0.1691, ndcg@10:0.2302, ndcg@5:0.1673
at epoch 3 , train time: 374.2 eval time: 60.7
at epoch 4
train info: logloss loss:0.5581357621793019
eval info: auc:0.5875, group_auc:0.5593, mean_mrr:0.1758, ndcg@10:0.2405, ndcg@5:0.1816
at epoch 4 , train time: 374.4 eval time: 60.5
at epoch 5
train info: logloss loss:0.5365664201267695
eval info: auc:0.6073, group_auc:0.6005, mean_mrr:0.2026, ndcg@10:0.2727, ndcg@5:0.2057
at epoch 5 , train time: 375.1 eval time: 60.4
at epoch 6
train info: logloss loss:0.5157902610756

# **Evaluate the DKN model**

In [ ]:
res = model.run_eval(test_file)
print(res)

{'auc': 0.6016, 'group_auc': 0.592, 'mean_mrr': 0.2008, 'ndcg@5': 0.2035, 'ndcg@10': 0.2727}


In [ ]:
sb.glue("res", res)